![alt text](./pageheader_rose2_babies.jpg)

# Processing of ventilator recordings using `ventiliser` package

#### Author:  Dr Gusztav Belteki and David Chong

This notebook runs the Ventiliser package on a ventilator recording (_DG007_) downloaded from the Draeger Babylog VN500 neonatal ventilator. 

This the paper describing _Ventiliser_: David Chong, Colin J Morley & Gusztav Belteki: **Computational analysis of neonatal ventilator waveforms and loops**. *Pediatric Research*, in press. 

The _Ventiliser_ Python package presented in the paper can be downloaded from [here](https://github.com/barrinalo/Ventiliser).

With questions and for more information please contact us: gbelteki@aol.com (Gusztav Belteki), dtwc3@cam.ac.uk (David Chong)

In [ ]:
from ventiliser.Draeger import Draeger
import time
import datetime
import os
import pandas as pd

# Suppress chained assignment warning, it does not affect the result
pd.set_option('mode.chained_assignment', None)

In [ ]:
# different set of recordings are in different folders

DRIVE = 'ELEMENTS'
# Directory on external drive to read the ventilation data from
DIR_READ_1 = '/Volumes/%s/Raw_data/Draeger/service_evaluation_old' % DRIVE
DIR_READ_2 = '/Volumes/%s/Raw_data/Draeger/service_evaluation_new' % DRIVE
DIR_READ_3 = '/Volumes/%s/Raw_data/Draeger/ventilation_CO2_elimination' % DRIVE

In [ ]:
# Recordings to run the pipeline on
recordings_selected = [ 'DG007']

In [ ]:
# Constants

freq = 100 
# Sampling rate (Hz)

flow_unit_converter = lambda x : x * 1000 / 60 / freq
# freq is incorporated in flow_unit_converter, because it assumes the data from the ventilator is in L/min. 
# Each datapoint is converted to mL/sample point so that when one integrates the flows 
# in each breath one will get the volume in mL. 

correction_window = 6000
# correction window is the window size (in data points) to perform averaging.

def run_pipeline(recording):
    
    x = int(recording.split('_')[0][-3:].lstrip('0'))
    if x <= 60:
        PATH = DIR_READ_1
    elif 60 < x <= 111:
        PATH = DIR_READ_2
    else:
        PATH = DIR_READ_3
    
    DATA_DIR = os.path.join(PATH, recording)

    tags = []
    for file in os.listdir(DATA_DIR):
        if "_fast_Unknown.csv" in file: # Files containing 100 Hz data have this tag
            tags.append(file)
    print(tags, '\n')

    for tag in tags:
        print("Processing " + tag)
        print("Start time:", datetime.datetime.now())
    
        # Instantiate pipeline and load data and settings
        print("Loading data and settings")
        pipeline = Draeger()
        # Columns containing absolute time, pressure, flow, respectively
        pipeline.load_data(os.path.join(DATA_DIR, tag), [0,4,5], 
            correction_window = correction_window, flow_unit_converter=flow_unit_converter)
        if os.path.exists(os.path.join(DATA_DIR, tag[:-17] + "_slow_Setting.csv")):
            pipeline.load_mapper_settings(os.path.join(DATA_DIR, tag[:-17] + "_slow_Setting.csv"))
            
        # Process
        pipeline.process()

In [ ]:
for recording in recordings_selected:
    print(recording, '\n')
    run_pipeline(recording)
    print('-' * 40)
    print('\n')